In [1]:
import findspark
findspark.init

<function findspark.init(spark_home=None, python_path=None, edit_rc=False, edit_profile=False)>

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("HousePricePredict").getOrCreate() 

23/01/30 12:08:28 WARN Utils: Your hostname, subbu resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/01/30 12:08:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/30 12:08:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [31]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext

from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col

from pyspark.ml.regression import LinearRegression
from pyspark.mllib.evaluation import RegressionMetrics

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, CrossValidatorModel
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler

In [6]:
stack_overflow_data = 'data.json'
df = spark.read.json(stack_overflow_data)
df.persist()

23/01/30 12:22:11 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


DataFrame[1stFlrSF: string, 2ndFlrSF: string, 3SsnPorch: string, Alley: string, BedroomAbvGr: string, BldgType: string, BsmtCond: string, BsmtExposure: string, BsmtFinSF1: string, BsmtFinSF2: string, BsmtFinType1: string, BsmtFinType2: string, BsmtFullBath: string, BsmtHalfBath: string, BsmtQual: string, BsmtUnfSF: string, CentralAir: string, Condition1: string, Condition2: string, Electrical: string, EnclosedPorch: string, ExterCond: string, ExterQual: string, Exterior1st: string, Exterior2nd: string, Fence: string, FireplaceQu: string, Fireplaces: string, Foundation: string, FullBath: string, Functional: string, GarageArea: string, GarageCars: string, GarageCond: string, GarageFinish: string, GarageQual: string, GarageType: string, GarageYrBlt: string, GrLivArea: string, HalfBath: string, Heating: string, HeatingQC: string, HouseStyle: string, Id: string, KitchenAbvGr: string, KitchenQual: string, LandContour: string, LandSlope: string, LotArea: string, LotConfig: string, LotFrontage

In [9]:
df.toPandas()

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,...,SaleType,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,Utilities,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
0,1402,823,0,NA,4,1Fam,TA,No,1129,0,...,WD,0,Pave,7,1390,AllPub,198,1992,1993,2010


In [10]:
df.printSchema()

root
 |-- 1stFlrSF: string (nullable = true)
 |-- 2ndFlrSF: string (nullable = true)
 |-- 3SsnPorch: string (nullable = true)
 |-- Alley: string (nullable = true)
 |-- BedroomAbvGr: string (nullable = true)
 |-- BldgType: string (nullable = true)
 |-- BsmtCond: string (nullable = true)
 |-- BsmtExposure: string (nullable = true)
 |-- BsmtFinSF1: string (nullable = true)
 |-- BsmtFinSF2: string (nullable = true)
 |-- BsmtFinType1: string (nullable = true)
 |-- BsmtFinType2: string (nullable = true)
 |-- BsmtFullBath: string (nullable = true)
 |-- BsmtHalfBath: string (nullable = true)
 |-- BsmtQual: string (nullable = true)
 |-- BsmtUnfSF: string (nullable = true)
 |-- CentralAir: string (nullable = true)
 |-- Condition1: string (nullable = true)
 |-- Condition2: string (nullable = true)
 |-- Electrical: string (nullable = true)
 |-- EnclosedPorch: string (nullable = true)
 |-- ExterCond: string (nullable = true)
 |-- ExterQual: string (nullable = true)
 |-- Exterior1st: string (nullabl

In [34]:
df = df.withColumn("LotFrontage", F.when(F.col("LotFrontage") == 'NA',70).otherwise(F.col("LotFrontage"))) \
.withColumn("LotFrontage", F.col("LotFrontage").cast("int")) \
.withColumn("MasVnrArea", F.when(F.col("MasVnrArea") == 'NA',104).otherwise(F.col("MasVnrArea"))) \
.withColumn("MasVnrArea", F.col("MasVnrArea").cast("int")) \
.withColumn("GarageYrBlt", F.when(F.col("GarageYrBlt") == 'NA',1979).otherwise(F.col("GarageYrBlt"))) \
.withColumn("GarageYrBlt", F.col("GarageYrBlt").cast("int")) \
.withColumn("BsmtFinSF1", F.when(F.col("BsmtFinSF1") == 'NA',441).otherwise(F.col("BsmtFinSF1"))) \
.withColumn("BsmtFinSF1", F.col("BsmtFinSF1").cast("int")) \
.withColumn("BsmtFinSF2", F.when(F.col("BsmtFinSF2") == 'NA',50).otherwise(F.col("BsmtFinSF2"))) \
.withColumn("BsmtFinSF2", F.col("BsmtFinSF2").cast("int")) \
.withColumn("BsmtUnfSF", F.when(F.col("BsmtUnfSF") == 'NA',561).otherwise(F.col("BsmtUnfSF"))) \
.withColumn("BsmtUnfSF", F.col("BsmtUnfSF").cast("int")) \
.withColumn("TotalBsmtSF", F.when(F.col("TotalBsmtSF") == 'NA',1052).otherwise(F.col("TotalBsmtSF"))) \
.withColumn("TotalBsmtSF", F.col("TotalBsmtSF").cast("int")) \
.withColumn("BsmtFullBath", F.when(F.col("BsmtFullBath") == 'NA',0.43).otherwise(F.col("BsmtFullBath"))) \
.withColumn("BsmtFullBath", F.col("BsmtFullBath").cast("float")) \
.withColumn("BsmtHalfBath", F.when(F.col("BsmtHalfBath") == 'NA',0.06).otherwise(F.col("BsmtHalfBath"))) \
.withColumn("BsmtHalfBath", F.col("BsmtHalfBath").cast("float")) \
.withColumn("GarageCars", F.when(F.col("GarageCars") == 'NA',2).otherwise(F.col("GarageCars"))) \
.withColumn("GarageCars", F.col("GarageCars").cast("int")) \
.withColumn("GarageArea", F.when(F.col("GarageArea") == 'NA',473).otherwise(F.col("GarageArea"))) \
.withColumn("GarageArea", F.col("GarageArea").cast("int"))

In [20]:
print(len(df.columns))

80


In [36]:
df.printSchema()

root
 |-- 1stFlrSF: string (nullable = true)
 |-- 2ndFlrSF: string (nullable = true)
 |-- 3SsnPorch: string (nullable = true)
 |-- Alley: string (nullable = true)
 |-- BedroomAbvGr: string (nullable = true)
 |-- BldgType: string (nullable = true)
 |-- BsmtCond: string (nullable = true)
 |-- BsmtExposure: string (nullable = true)
 |-- BsmtFinSF1: integer (nullable = true)
 |-- BsmtFinSF2: integer (nullable = true)
 |-- BsmtFinType1: string (nullable = true)
 |-- BsmtFinType2: string (nullable = true)
 |-- BsmtFullBath: float (nullable = true)
 |-- BsmtHalfBath: float (nullable = true)
 |-- BsmtQual: string (nullable = true)
 |-- BsmtUnfSF: integer (nullable = true)
 |-- CentralAir: string (nullable = true)
 |-- Condition1: string (nullable = true)
 |-- Condition2: string (nullable = true)
 |-- Electrical: string (nullable = true)
 |-- EnclosedPorch: string (nullable = true)
 |-- ExterCond: string (nullable = true)
 |-- ExterQual: string (nullable = true)
 |-- Exterior1st: string (nullab

In [56]:
from pyspark.ml import PipelineModel

pipemodel = PipelineModel.load("/home/subbu/Documents/model")

prediction = pipemodel.transform(df)

prediction.toPandas()

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,...,GarageType_ohe,GarageFinish_ohe,GarageQual_ohe,GarageCond_ohe,PavedDrive_ohe,SaleType_ohe,SaleCondition_ohe,unscaled_features,features,prediction
0,1402,823,0,NA,4,1Fam,TA,No,1129,0,...,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0)","(60.0, 88.0, 11134.0, 8.0, 5.0, 1992.0, 1993.0...","(1.4205519848118042, 3.8965587486001727, 1.119...",336161.618352


In [57]:
prediction.select('prediction').show()

+-----------------+
|       prediction|
+-----------------+
|336161.6183518471|
+-----------------+



In [37]:
catcols=['MSZoning', 'Street', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'BldgType', 'HouseStyle', 'RoofStyle', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'SaleType', 'SaleCondition']

In [38]:
numcols=['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']

In [45]:
from pyspark.sql.types import IntegerType

In [48]:
df = df.withColumn("MSSubClass", col("MSSubClass").cast(IntegerType()))

In [49]:
df.printSchema()

root
 |-- 1stFlrSF: string (nullable = true)
 |-- 2ndFlrSF: string (nullable = true)
 |-- 3SsnPorch: string (nullable = true)
 |-- Alley: string (nullable = true)
 |-- BedroomAbvGr: string (nullable = true)
 |-- BldgType: string (nullable = true)
 |-- BsmtCond: string (nullable = true)
 |-- BsmtExposure: string (nullable = true)
 |-- BsmtFinSF1: integer (nullable = true)
 |-- BsmtFinSF2: integer (nullable = true)
 |-- BsmtFinType1: string (nullable = true)
 |-- BsmtFinType2: string (nullable = true)
 |-- BsmtFullBath: float (nullable = true)
 |-- BsmtHalfBath: float (nullable = true)
 |-- BsmtQual: string (nullable = true)
 |-- BsmtUnfSF: integer (nullable = true)
 |-- CentralAir: string (nullable = true)
 |-- Condition1: string (nullable = true)
 |-- Condition2: string (nullable = true)
 |-- Electrical: string (nullable = true)
 |-- EnclosedPorch: string (nullable = true)
 |-- ExterCond: string (nullable = true)
 |-- ExterQual: string (nullable = true)
 |-- Exterior1st: string (nullab

In [54]:
for col_name in numcols:
    print(col_name)
    df = df.withColumn(col_name, col(col_name).cast(IntegerType()))

MSSubClass
LotFrontage
LotArea
OverallQual
OverallCond
YearBuilt
YearRemodAdd
MasVnrArea
BsmtFinSF1
BsmtFinSF2
BsmtUnfSF
TotalBsmtSF
1stFlrSF
2ndFlrSF
LowQualFinSF
GrLivArea
BsmtFullBath
BsmtHalfBath
FullBath
HalfBath
BedroomAbvGr
KitchenAbvGr
TotRmsAbvGrd
Fireplaces
GarageYrBlt
GarageCars
GarageArea
WoodDeckSF
OpenPorchSF
EnclosedPorch
3SsnPorch
ScreenPorch
PoolArea
MiscVal
MoSold
YrSold


In [55]:
df.printSchema()

root
 |-- 1stFlrSF: integer (nullable = true)
 |-- 2ndFlrSF: integer (nullable = true)
 |-- 3SsnPorch: integer (nullable = true)
 |-- Alley: string (nullable = true)
 |-- BedroomAbvGr: integer (nullable = true)
 |-- BldgType: string (nullable = true)
 |-- BsmtCond: string (nullable = true)
 |-- BsmtExposure: string (nullable = true)
 |-- BsmtFinSF1: integer (nullable = true)
 |-- BsmtFinSF2: integer (nullable = true)
 |-- BsmtFinType1: string (nullable = true)
 |-- BsmtFinType2: string (nullable = true)
 |-- BsmtFullBath: integer (nullable = true)
 |-- BsmtHalfBath: integer (nullable = true)
 |-- BsmtQual: string (nullable = true)
 |-- BsmtUnfSF: integer (nullable = true)
 |-- CentralAir: string (nullable = true)
 |-- Condition1: string (nullable = true)
 |-- Condition2: string (nullable = true)
 |-- Electrical: string (nullable = true)
 |-- EnclosedPorch: integer (nullable = true)
 |-- ExterCond: string (nullable = true)
 |-- ExterQual: string (nullable = true)
 |-- Exterior1st: strin